<a href="https://colab.research.google.com/github/lauren1turner/DS4002_LAM/blob/main/prelim_TF_IDF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer, ENGLISH_STOP_WORDS
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import seaborn as sns

# 1. Load the CSV file
df = pd.read_csv('DC_clean.csv')

# Ensure the text data is treated as strings (adjust the column name if necessary)
documents = df['Lyrics'].astype(str)

# 2. Define custom stop words
custom_stop_words = {'chorus', 'uh', 'verse', 'like'}

# Combine the custom stop words with the default English stop words and convert to a list
combined_stop_words = list(ENGLISH_STOP_WORDS.union(custom_stop_words))

# 3. Apply TfidfVectorizer to the text data using the combined stop words
vectorizer = TfidfVectorizer(stop_words=combined_stop_words, max_df=0.9, min_df=5)
tfidf_matrix = vectorizer.fit_transform(documents)
print("TF-IDF matrix shape:", tfidf_matrix.shape)

# 4. Analysis Part 1: Identify Top Features by Average TF-IDF Score
mean_tfidf = tfidf_matrix.mean(axis=0).A1  # Convert to a 1D NumPy array
top_indices = np.argsort(mean_tfidf)[::-1][:10]
top_features = [vectorizer.get_feature_names_out()[i] for i in top_indices]
print("Top 10 features by average TF-IDF score:", top_features)



TF-IDF matrix shape: (25, 78)
Top 10 features by average TF-IDF score: ['yeah', 'oh', 'know', 'just', 've', 'la', 'don', 'got', 'gettin', 'time']
